# 1 Preliminary

In [ ]:
cd ..

In [ ]:
pwd

In [ ]:
ls

## 1.1 Install

Install 3rd party tools : 

In [ ]:
# !sudo apt install unzip tree htop

Install various Librairies : 

In [ ]:
# !pip install -r requirements.txt >> pip.log
# !pip freeze >> pip.freeze

In [ ]:
!python -m spacy download en_core_web_sm >> spacy.log
!python -m spacy download en_core_web_md >> spacy.log
# !python -m spacy download en_core_web_lg >> spacy.log

## 1.2 Import 

In [ ]:
import os, sys, warnings
import pickle
from IPython.display import display

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import missingno as msno

from sklearn.base import *
from sklearn.preprocessing import *
from sklearn.impute import *
from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.ensemble import *
from sklearn.model_selection import *
from sklearn.pipeline import *
from sklearn.feature_extraction import *
from sklearn.dummy import *
from sklearn.feature_extraction.text import *

from lightgbm import *
from xgboost import *
from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.neighbors import *


import nltk
import wordcloud

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import gensim
import transformers

import nltk
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize import wordpunct_tokenize
import string

In [ ]:
# nlp = spacy.load('en_core_web_lg')

## 1.3 Graphs and Settings

In [ ]:
sns.set()

In [ ]:
# warnings.filterwarnings('ignore')
warnings.filterwarnings(action="once")

In [ ]:
DISPLAY = True

## 1.4 Data

In [ ]:
source = "./data/cleaned/"

In [ ]:
df = pd.read_csv(source + "df2.csv")
DF = df.copy()
df.head(2)

# 2 First tour 

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.dtypes.value_counts()

In [ ]:
tmp = df.isna().mean(axis=0)
tmp

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe()

In [ ]:
df.describe(include=object)

# 3 Prepare Data

In [ ]:
y = df.cat_1
y

In [ ]:
X = df.description

In [ ]:
def cv():
    return StratifiedShuffleSplit(n_splits=5, test_size=0.25)


cv()

# 4 Using Gensim

## 4.1 By Hand

In [ ]:
documents = df.description
documents[:10]

In [ ]:
# Tokenize and preprocess the documents using spaCy
tokenized_docs = [
    [
        token.lemma_
        for token in nlp(doc.lower())
        if not token.is_stop and not token.is_punct
    ]
    for doc in documents
]
tokenized_docs[:10]

In [ ]:
# Create tagged documents
tagged_docs = [
    TaggedDocument(words=doc, tags=[i]) for i, doc in enumerate(tokenized_docs)
]
tagged_docs[:10]

In [ ]:
# Train the Doc2Vec model
model = Doc2Vec(
    tagged_docs, vector_size=50, window=2, min_count=1, workers=4, epochs=100
)
model

In [ ]:
# Get the document vectors
doc_vectors = [model.infer_vector(doc) for doc in tokenized_docs]
print(doc_vectors)

In [ ]:
type(doc_vectors)

In [ ]:
len(doc_vectors)

In [ ]:
len(df)

In [ ]:
X = pd.DataFrame(doc_vectors)
X

In [ ]:
X.shape

In [ ]:
grid = GridSearchCV(
    LogisticRegression(), {}, cv=cv(), n_jobs=-1, return_train_score=True, verbose=1
)

grid.fit(X, y)

In [ ]:
display(grid.best_estimator_)

res = pd.DataFrame(grid.cv_results_)
cols = [i for i in res.columns if "split" not in i]
res = res.loc[:, cols]
res.round(2).sort_values("mean_test_score", ascending=False).head(10)

In [ ]:
pst = "passthrough"

pipeline = Pipeline(
    [
        ("preprocessor", pst),
        ("scaler", pst),
        ("reductor", pst),
        ("estimator", LogisticRegression()()),
    ]
)

pipeline

In [ ]:
param_grid = {
    "scaler": [
        "passthrough",
        StandardScaler(),
        QuantileTransformer(n_quantiles=100),
        MinMaxScaler(),
        Normalizer(),
    ],
    "reductor": [PCA()],
    "reductor__n_components": [0.7, 0.85, 0.9, 0.95, 0.99],
    "estimator": [RandomForestClassifier(), LGBMClassifier(), LogisticRegression()],
}
param_grid

In [ ]:
grid = GridSearchCV(
    pipeline, param_grid, cv=cv(), n_jobs=-1, return_train_score=True, verbose=1
)

grid.fit(X, y)

In [ ]:
display(grid.best_estimator_)

res = pd.DataFrame(grid.cv_results_)
cols = [i for i in res.columns if "split" not in i]
res = res.loc[:, cols]
res.round(2).sort_values("mean_test_score", ascending=False).head(10)

## 4.2 using Pipeline

In [ ]:
pst = "passthrough"

pipeline = Pipeline(
    [
        ("preprocessor", pst),
        ("scaler", pst),
        ("reductor", pst),
        ("estimator", RandomForestClassifier()),
    ]
)

pipeline

In [ ]:
class Doc2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vector_size=100, window=5, min_count=5, epochs=10):
        self.vector_size = vector_size
        self.window = window
        self.min_count = min_count
        self.epochs = epochs
        self.model = None

    def fit(self, X, y=None):
        tagged_data = [
            TaggedDocument(words=doc.split(), tags=[i]) for i, doc in enumerate(X)
        ]
        self.model = Doc2Vec(
            tagged_data,
            vector_size=self.vector_size,
            window=self.window,
            min_count=self.min_count,
            epochs=self.epochs,
        )
        return self

    def transform(self, X, y=None):
        return [self.model.infer_vector(doc.split()) for doc in X]

In [ ]:
param_grid = {"preprocessor": [Doc2VecTransformer()]}

In [ ]:
grid = GridSearchCV(
    pipeline, param_grid, cv=cv(), n_jobs=-1, return_train_score=True, verbose=1
)

grid.fit(tokenized_docs, y)

grid.best_estimator_

In [ ]:
display(grid.best_estimator_)

res = pd.DataFrame(grid.cv_results_)
cols = [i for i in res.columns if "split" not in i]
res = res.loc[:, cols]
res.round(2).sort_values("mean_test_score", ascending=False).head(10)

In [ ]:
# doc2vec_transformer = Doc2VecTransformer(vector_size=100, window=5, min_count=5, epochs=10)
# tfidf_vectorizer = TfidfVectorizer()
# logistic_regression = LogisticRegression()

In [ ]:
param_grid = {
    "preprocessor": [Doc2VecTransformer()],
    "preprocessor__vector_size": [100, 200, 300],
    "preprocessor__window": [5, 10, 15],
}

In [ ]:
grid = GridSearchCV(
    pipeline, param_grid, cv=cv(), n_jobs=-1, return_train_score=True, verbose=1
)

grid.fit(tokenized_docs, y)

grid.best_estimator_

In [ ]:
display(grid.best_estimator_)

res = pd.DataFrame(grid.cv_results_)
cols = [i for i in res.columns if "split" not in i]
res = res.loc[:, cols]
res.round(2).sort_values("mean_test_score", ascending=False).head(10)